
# Clustering

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.feature_extraction.text
import sklearn.cluster
import sklearn.decomposition
#from adjustText import adjust_text
import altair as alt

import sys
sys.path.insert(0, '../../scripts')
import util
import theme_desarroio

import matplotlib.pyplot as plt

In [30]:
theme=theme_desarroio.theme_desarroio()

Loading file:

In [4]:
#file = '../../data/clean/speech.pkl'
file = '../../data/clean/speech_init.pkl'

In [5]:
df = pd.read_pickle(file)

In [6]:
df

,filename,raw text,year,cleaned text,administration,president,year-president,tokenized_words,normalized_words,tokenized_sentences,tokenized_words_in_sentences,normalized_words_in_sentences
0,mensaje-1963-fbt.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1963,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Belaunde(1),Belaunde,1963-Belaunde,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, ar...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, a..."
1,mensaje-1969-jva.txt,"[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\...",1969,"MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, G...",Velasco,Velasco,1969-Velasco,"[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL,...","[mensaje, nación, presidente, perú, división, ...","[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, ...","[[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL...","[[mensaje, nación, presidente, perú, división,..."
2,mensaje-1976-fmb.txt,"[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ,\...",1976,"MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, ...",Morales Bermudez,Morales Bermudez,1976-Morales Bermudez,"[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL,...","[mensaje, nación, presidente, perú, división, ...","[MENSAJE A LA NACIÓN DEL PRESIDENTE DEL PERÚ, ...","[[MENSAJE, A, LA, NACIÓN, DEL, PRESIDENTE, DEL...","[[mensaje, nación, presidente, perú, división,..."
3,mensaje-1980-fbt.txt,"[ARQUITECTO FERNANDO BELAUNDE TERRY, \n, \n, A...",1980,"ARQUITECTO FERNANDO BELAUNDE TERRY, ANTE EL C...",Belaunde(2),Belaunde,1980-Belaunde,"[ARQUITECTO, FERNANDO, BELAUNDE, TERRY, ANTE, ...","[arquitecto, fernando, belaunde, terry, congre...","[ARQUITECTO FERNANDO BELAUNDE TERRY, ANTE EL ...","[[ARQUITECTO, FERNANDO, BELAUNDE, TERRY, ANTE,...","[[arquitecto, fernando, belaunde, terry, congr..."
4,mensaje-1985-ag.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1985,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Garcia(1),Garcia,1985-Garcia,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, do...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, d..."
5,mensaje-1990-af.txt,"[INGENIERO ALBERTO FUJIMORI FUJIMORI, \n, \n, ...",1990,"INGENIERO ALBERTO FUJIMORI FUJIMORI, ANTE EL ...",Fujimori(1),Fujimori,1990-Fujimori,"[INGENIERO, ALBERTO, FUJIMORI, FUJIMORI, ANTE,...","[ingeniero, alberto, fujimori, fujimori, congr...","[INGENIERO ALBERTO FUJIMORI FUJIMORI, ANTE EL...","[[INGENIERO, ALBERTO, FUJIMORI, FUJIMORI, ANTE...","[[ingeniero, alberto, fujimori, fujimori, cong..."
6,mensaje-1995-af.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,1995,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Fujimori(2),Fujimori,1995-Fujimori,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, in...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, i..."
7,mensaje-2000-af.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,2000,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Fujimori(2),Fujimori,2000-Fujimori,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, in...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú, i..."
8,mensaje-2000-vp-noviembre.txt,[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,2000,MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PERÚ...,Paniagua,Paniagua,2000-Paniagua,"[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DEL...","[mensaje, presidente, constitucional, perú, do...",[MENSAJE DEL PRESIDENTE CONSTITUCIONAL DEL PER...,"[[MENSAJE, DEL, PRESIDENTE, CONSTITUCIONAL, DE...","[[mensaje, presidente, constitucional, perú

## Analysis description

The main purpose of this analysis is to explore the associations and patterns present in the word frequency of each speech. Specifically, we would like to assess if speeches within the same cluster signal similarities in political discourses or closeness in the left-right political spectrum.

To visualize the result, we will generate the first two components of the Principal Component Analysis (PCA) of the tf-idf matrix for each speech. Then we will do a scatter plot of each speech and its position according to these two principal components to see how are the clusters distributed.

## K-means

In [7]:
tf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=5, norm='l2')

In [8]:
df['normalized_words_concat'] = df['normalized_words'].apply(lambda x: ' '.join(x))

In [9]:
#col = 'cleaned text'
col = 'normalized_words_concat'

In [10]:
tf_vects = tf_vectorizer.fit_transform(df[col])

In [11]:
terms = tf_vectorizer.get_feature_names()

In [12]:
#n_clusters = [3, 4, 5, 6, 7, 8, 9, 10]
n_clusters = [2,3,4]

In [13]:
for n in n_clusters:
    km = sklearn.cluster.KMeans(n_clusters = n, init='k-means++')
    km.fit(tf_vects)
    df['kmeans' + str(n)] = km.labels_

## PCA

In [14]:
pca = sklearn.decomposition.PCA(n_components = 2).fit(tf_vects.toarray())
reduced_pca = pca.transform(tf_vects.toarray())

In [15]:
pca_df = pd.DataFrame()
pca_df['president'] = df['president']
pca_df['pca1'] = reduced_pca[:,0]
pca_df['pca2'] = reduced_pca[:,1]

for num in n_clusters:
    pca_df['clusters_'+str(num)] = df['kmeans'+str(num)]

In [31]:
points = alt.Chart(pca_df).mark_circle(
    size=60
    ).encode(
    x = alt.X('pca1:Q', axis=alt.Axis(labels=False), title=None),
    y = alt.Y('pca2:Q', axis=alt.Axis(labels=False), title=None),
    color = 'clusters_3:N'
    ).properties(
    title={
        "text": ["Análisis de clusters"], 
        "subtitle": ["Espacios semánticos en los discursos inaugurales",
                     "Componentes en PCA: 2"],
        "subtitleFont": "Sanchez",
    }
)

text = points.mark_text(
    align = 'center',
    baseline = 'middle',
    dy = -10
    ).encode(
    text = 'president'
    )

(points + text).configure_axis(
    grid = False,
    ticks = False
)

alt.LayerChart(...)